<a href="https://colab.research.google.com/github/catalinapesquet/everything-training/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data&Data Internship Assignment

This is the notebook that you should complete.

Please ensure that your code is clean, modular, and easy to follow. Feel free to create your own modules and import them into the notebook if it helps with clarity and structure.

 Feel free to add or delete cells in the notebook depending on you needs.

In [1]:
# Imports (import the libraries you need)

# CRUD operations libraries
import sqlite3
from sqlite3 import Error

In [2]:
# Web Scrapping libraries
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 21.6 MB/s eta 0:00:00


In [30]:
!pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 4.6 MB/s eta 0:00:00


In [44]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import tempfile
import re
import unidecode

# Entity Matching
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import numpy as np

## Part 1: CRUD Operations

Either use SQLThunder or implement a class that supports the four basic CRUD operations: **Create, Read, Update, and Delete**.

SQLThunder docs: https://sqlthunder.readthedocs.io/en/latest/

In [4]:
# Class implementation or SQLThunder DBClient initialization

class CRUDOperations:
    def __init__(self, db_file):
        """Initialize the database connection"""
        self.db_file = db_file
        self.conn = None
        try:
            self.conn = sqlite3.connect(db_file)
        except Error as e:
            print(e)

    def create_table(self):
        """Create a table for demonstration"""
        sql_create_table = """CREATE TABLE IF NOT EXISTS Products (
                                id integer PRIMARY KEY,
                                name text NOT NULL
                            );"""
        try:
            c = self.conn.cursor()
            c.execute(sql_create_table)
        except Error as e:
            print(e)

    def create(self, name):
        """Create a new product"""
        sql = '''INSERT INTO Products(name)
                 VALUES(?)'''
        cur = self.conn.cursor()
        cur.execute(sql, (name,))
        self.conn.commit()
        return cur.lastrowid

    def read(self):
        """Query all rows in the Products table"""
        cur = self.conn.cursor()
        cur.execute("SELECT * FROM Products")
        rows = cur.fetchall()
        return rows

    def update(self, product_id, name):
        """Update name of a product"""
        sql = '''UPDATE Products
                 SET name = ?
                 WHERE id = ?'''
        cur = self.conn.cursor()
        cur.execute(sql, (name, product_id))
        self.conn.commit()

    def delete(self, product_id):
        """Delete a product by id"""
        sql = 'DELETE FROM Products WHERE id=?'
        cur = self.conn.cursor()
        cur.execute(sql, (product_id,))
        self.conn.commit()

    def __del__(self):
        """Destructor to close the database connection"""
        if self.conn:
            self.conn.close()
    def drop_table(self):
      sql = "DROP TABLE IF EXISTS Products"
      cur = self.conn.cursor()
      cur.execute(sql)
      self.conn.commit()

Create a simple table for demonstration purposes, and perform the four operations using just one row (the content is up to you)

In [5]:
# Create table
db_file = 'demo_database.db'
crud = CRUDOperations(db_file)
crud.drop_table()
crud.create_table()

In [6]:
# Create operation (i.e. insert)
product_id = crud.create("Sample Product")
print(f"Created product with ID: {product_id}")

Created product with ID: 1


In [7]:
# Read operation
products = crud.read()
print("Products in database:")
for product in products:
    print(product)

Products in database:
(1, 'Sample Product')


In [8]:
# Update operation
crud.update(2, "Updated Product")
print("Product updated.")
products = crud.read()
print("Products in database after update:")
for product in products:
    print(product)

Product updated.
Products in database after update:
(1, 'Sample Product')


In [9]:
# Delete operation
crud.delete(1)
print("Product deleted.")
products = crud.read()
print("Products in database after deletion:")
for product in products:
    print(product)

Product deleted.
Products in database after deletion:


## Part 2: Web Scraping

Scrape data from profumeriaideale.com website for Valmont.

**Note**: Selenium is cool ;)

In [10]:
# temporary directory for user data
temp_profile = tempfile.mkdtemp()

options = Options()
options.add_argument('--headless')  # Exécuter en mode headless
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(options=options)

In [11]:
def scrape_valmont_page(url):
    driver = webdriver.Chrome(options=options)
    driver.get(url)

    wait = WebDriverWait(driver, 10)
    try:
        products = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'article.product-miniature')))
    except:
        print("No product found or timeout")
        driver.quit()
        return []

    data = []
    for prod in products:
        try:
            title_elem = prod.find_element(By.CSS_SELECTOR, 'h2.product-title a')
            product_name = title_elem.text
            # product_url = title_elem.get_attribute('href')
            capacity_match = re.search(r'(\d+\s?ml)', product_name, re.IGNORECASE)
            capacity = capacity_match.group(1) if capacity_match else None

            data.append({
                'product_name': product_name,
                'capacity': capacity,
                # 'product_url': product_url
            })
        except Exception as e:
            print("Error extracting the product:", e)

    driver.quit()
    return data

In [12]:
all_products = []
for page_num in range(1, 10):
    url = f"https://www.profumeriaideale.com/brand/190-valmont?page={page_num}"
    print(f"Scraping page {page_num}: {url}")
    products = scrape_valmont_page(url)
    if not products:
        print("Empty page")
        break
    all_products.extend(products)
    time.sleep(2)

print(f"Total number of extracted products: {len(all_products)}")

Scraping page 1: https://www.profumeriaideale.com/brand/190-valmont?page=1
Scraping page 2: https://www.profumeriaideale.com/brand/190-valmont?page=2
Scraping page 3: https://www.profumeriaideale.com/brand/190-valmont?page=3
Scraping page 4: https://www.profumeriaideale.com/brand/190-valmont?page=4
Scraping page 5: https://www.profumeriaideale.com/brand/190-valmont?page=5
No product found or timeout
Empty page
Total number of extracted products: 88


In [13]:
# More scraping ?


In [14]:
# Even more scraping ?


## Part 3: Cleaning and Inserting Data

Visualizing data:


In [15]:
# Visualizing
all_products

[{'product_name': 'Valmont Aqua Falls Acqua Struccante Istantanea 150 ml',
  'capacity': '150 ml'},
 {'product_name': 'Valmont Bi-Falls 60 ml', 'capacity': '60 ml'},
 {'product_name': 'Valmont Blooming Ballet Eau De Toilette Donna 100ml',
  'capacity': '100ml'},
 {'product_name': 'Valmont Body 24 Hour – Crema Corpo Anti-Aging 150 ml',
  'capacity': '150 ml'},
 {'product_name': 'Valmont Bubble Falls Mousse Detergente Equilibrante 150 Ml',
  'capacity': '150 Ml'},
 {'product_name': "Valmont calendario dell'avvento holidays in neverland",
  'capacity': None},
 {'product_name': 'Valmont Collezione Privata Jazzy Twist Eau De Parfum Floral Gourmand Unisex 100 ml',
  'capacity': '100 ml'},
 {'product_name': 'Valmont Collezione Privata Just Bloom Eau De Parfum 100 ml',
  'capacity': '100 ml'},
 {'product_name': 'Valmont Collezione Privata Lady Code Eau De Parfum 100 ml',
  'capacity': '100 ml'},
 {'product_name': 'Valmont Collezione Privata Private Mind Eau De Parfum Floral Leather Unisex 100 

In [16]:
# Cleaning
def clean_capacity(capacity):
    if capacity is None:
        return None
    cleaned = capacity.strip().lower().replace(' ', '')
    # Corriger les fautes typiques (ex : 'ml', 'ML', 'Ml')
    if 'ml' in cleaned:
        return cleaned.replace('ml', '') + ' ml'
    return cleaned

def clean_product_name(name):
    name = name.strip()
    name = re.sub(r'\s+', ' ', name)  # multiple spaces
    name = re.sub(r'\bml\b', 'ml', name, flags=re.IGNORECASE)  # mL'
    name = re.sub(r'\s+ml', ' ml', name, flags=re.IGNORECASE)
    name = name.rstrip(' .')  # erase ponctuation
    return name


In [17]:
cleaned_products = []

for item in all_products:  # raw_products = ton dict original
    cleaned = {
        'product_name': clean_product_name(item['product_name']),
        'capacity': clean_capacity(item['capacity'])
    }
    cleaned_products.append(cleaned)

In [18]:
cleaned_products

[{'product_name': 'Valmont Aqua Falls Acqua Struccante Istantanea 150 ml',
  'capacity': '150 ml'},
 {'product_name': 'Valmont Bi-Falls 60 ml', 'capacity': '60 ml'},
 {'product_name': 'Valmont Blooming Ballet Eau De Toilette Donna 100ml',
  'capacity': '100 ml'},
 {'product_name': 'Valmont Body 24 Hour – Crema Corpo Anti-Aging 150 ml',
  'capacity': '150 ml'},
 {'product_name': 'Valmont Bubble Falls Mousse Detergente Equilibrante 150 ml',
  'capacity': '150 ml'},
 {'product_name': "Valmont calendario dell'avvento holidays in neverland",
  'capacity': None},
 {'product_name': 'Valmont Collezione Privata Jazzy Twist Eau De Parfum Floral Gourmand Unisex 100 ml',
  'capacity': '100 ml'},
 {'product_name': 'Valmont Collezione Privata Just Bloom Eau De Parfum 100 ml',
  'capacity': '100 ml'},
 {'product_name': 'Valmont Collezione Privata Lady Code Eau De Parfum 100 ml',
  'capacity': '100 ml'},
 {'product_name': 'Valmont Collezione Privata Private Mind Eau De Parfum Floral Leather Unisex 100

Insertion using class/SQLThunder form Part 1

In [19]:
# Insert data in your local db (you might need to create a new table)

crud = CRUDOperations("produits_valmont.db")

# Create table
crud.drop_table()
crud.create_table()

# Insert product
for p in cleaned_products:
    crud.create(p['product_name'])

# Read and check
all_products = crud.read()
for prod in all_products:
    print(prod)


Exception ignored in: <function CRUDOperations.__del__ at 0x7df40436d300>
Traceback (most recent call last):
  File "/tmp/ipython-input-95169380.py", line 60, in __del__
sqlite3.ProgrammingError: SQLite objects created in a thread can only be used in that same thread. The object was created in thread id 138488185229312 and this is thread id 138487495898688.


(1, 'Valmont Aqua Falls Acqua Struccante Istantanea 150 ml')
(2, 'Valmont Bi-Falls 60 ml')
(3, 'Valmont Blooming Ballet Eau De Toilette Donna 100ml')
(4, 'Valmont Body 24 Hour – Crema Corpo Anti-Aging 150 ml')
(5, 'Valmont Bubble Falls Mousse Detergente Equilibrante 150 ml')
(6, "Valmont calendario dell'avvento holidays in neverland")
(7, 'Valmont Collezione Privata Jazzy Twist Eau De Parfum Floral Gourmand Unisex 100 ml')
(8, 'Valmont Collezione Privata Just Bloom Eau De Parfum 100 ml')
(9, 'Valmont Collezione Privata Lady Code Eau De Parfum 100 ml')
(10, 'Valmont Collezione Privata Private Mind Eau De Parfum Floral Leather Unisex 100 ml')
(11, 'Valmont Deto2X Cream Crema Ossigenante Detossinante 45 ml')
(12, 'Valmont Deto2X Eye Crema Contorno Occhi Vitalità 12 ml')
(13, 'Valmont Deto2X Pack Bubble Mask Ossigenante 6 Pezzi Da 10 ml')
(14, 'Valmont Eau De Toilette 100 ml Palazzo Nobile Cutie')
(15, 'Valmont Elixir Serum Precieux 30 ml')
(16, 'Valmont Eye Instant Stress Relieving Mask B

## Entity Matching

Try to match each scraped data to official ref if it exists.

In [46]:
# Load data
refs = pd.read_excel("refs.xlsx")

conn = sqlite3.connect("produits_valmont.db")

offers = pd.read_sql_query("SELECT * FROM products", conn)

# Facultatif : renommer les colonnes si besoin
offers.columns = ['offer_uid', 'title']  # ou ['id', 'title'] selon les noms exacts dans ta table

# Vérification
print(offers.head())

   offer_uid                                              title
0          1  Valmont Aqua Falls Acqua Struccante Istantanea...
1          2                             Valmont Bi-Falls 60 ml
2          3  Valmont Blooming Ballet Eau De Toilette Donna ...
3          4  Valmont Body 24 Hour – Crema Corpo Anti-Aging ...
4          5  Valmont Bubble Falls Mousse Detergente Equilib...


In [47]:
offers['clean_title'] = offers['title'].apply(clean_product_name)
refs['clean_name'] = refs['product_name'].apply(clean_product_name)

In [48]:
# Match
# Combine vocabulary
all_titles = pd.concat([offers['clean_title'], refs['clean_name']])

# Vectorisation TF-IDF
vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1,2), min_df=2)
vectorizer.fit(all_titles)
tfidf_matrix = vectorizer.fit_transform(all_titles)

offers_vec = vectorizer.transform(offers['clean_title'])
refs_vec = vectorizer.transform(refs['clean_name'])

similarity_matrix = cosine_similarity(offers_vec, refs_vec)

best_match_indices = np.argmax(similarity_matrix, axis=1)

best_match_scores = np.max(similarity_matrix, axis=1)

# Create DataFrame (results)
matched_skus = refs.iloc[best_match_indices]['SKU'].values

submission_df = pd.DataFrame({
    'offer_uid': offers['offer_uid'],
    'SKU': matched_skus,
    'similarity': best_match_scores
})

# Create DataFrame (results)

matched_skus = refs.iloc[best_match_indices]['SKU'].values

submission_df = pd.DataFrame({
    'offer_uid': offers['offer_uid'],
    'SKU': matched_skus,
    'similarity': best_match_scores
})

submission_df.to_csv("submission.csv", index=False)
print(submission_df.head())

   offer_uid      SKU  similarity
0          1   705045    0.627203
1          2   705047    0.941690
2          3   803004    0.558310
3          4  7058100    0.676028
4          5   705046    0.699925


In [49]:
refs['SKU'] = refs['SKU'].astype(str).str.strip()
submission_df['SKU'] = submission_df['SKU'].astype(str).str.strip()

# Sort by similarity
worst_matches = submission_df.sort_values(by='similarity').head(20)

print("=== 10 worst matches ===")

for idx, row in worst_matches.iterrows():
    offer_uid = row['offer_uid']
    sku = row['SKU']
    similarity = row['similarity']

    offer_text = offers.loc[offers['offer_uid'] == offer_uid, 'title'].values
    offer_text = offer_text[0] if len(offer_text) > 0 else "N/A"

    # Find product name in ref
    if sku not in refs['SKU'].values:
        ref_text = "SKU non trouvé dans refs"
    else:
        ref_text = refs.loc[refs['SKU'] == sku, 'product_name'].values[0]
    print(f"Offer ID {offer_uid} (Similarity: {similarity:.3f}):\n  Offer: {offer_text}\n  Reference: {ref_text}\n")



=== 10 worst matches ===
Offer ID 6 (Similarity: 0.000):
  Offer: Valmont calendario dell'avvento holidays in neverland
  Reference: Alessandrite I

Offer ID 14 (Similarity: 0.197):
  Offer: Valmont Eau De Toilette 100 ml Palazzo Nobile Cutie
  Reference: Cutie Pear

Offer ID 35 (Similarity: 0.273):
  Offer: Valmont Lumicity Trattamento Anti-Age Illuminante Spf50 - 30ml
  Reference: LUMICITY SPF 50+ 

Offer ID 75 (Similarity: 0.276):
  Offer: Valmont Storie Veneziane Rosso I Extrait De Parfum Floral Oriental Unisex 100 ml
  Reference: Rosso I

Offer ID 70 (Similarity: 0.280):
  Offer: Valmont Storie Veneziane Alessandrite I Extrait De Parfum Aldehydic Floral Unisex
  Reference: Alessandrite I

Offer ID 71 (Similarity: 0.282):
  Offer: Valmont Storie Veneziane Alessandrite I'Extrait De Parfum Aldehydic 100 ml
  Reference: Alessandrite I

Offer ID 77 (Similarity: 0.328):
  Offer: Valmont Storie Veneziane Zafferano I Extrait De Parfum Floral-Oriental Especiado Unisex
  Reference: Zafferan

In [23]:
# Lots of Matching


In [24]:
# Now you can add as manyyyy cells as you want to match


### Output Excel

Please save your result in an excel in the format mentioned in the README.MD and submit it to Kaggle. Don't forget to send the repo by email.